In [1]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import psycopg2
import time

In [2]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [ ]:
def movie_data_transformation_pipeline(wiki_movies_json_file, kaggle_metadata_csv_file, movielens)
    ## file: /Users/priyankasenapati/Desktop/Classwork/Movies-ETL/wikipedia.movies.json
    ## /Users/priyankasenapati/Desktop/Classwork/Movies-ETL/movies_metadata.csv
    ##

    # Import Wikipedia Movies
    ## file_dir = '/Users/priyankasenapati/Desktop/Classwork/Movies-ETL/'
    ## with open(f'{file_dir}/wikipedia.movies.json', mode='r') as file:
    ## file: /Users/priyankasenapati/Desktop/Classwork/Movies-ETL/wikipedia.movies.json
    with open(f'{wiki_movies_json_file}', mode='r') as file:
    wiki_movies_raw = json.load(file)
    ## len(wiki_movies_raw)
    
    # Import Kaggle Metadata
    ## kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv')
    ## file: /Users/priyankasenapati/Desktop/Classwork/Movies-ETL/movies_metadata.csv
    kaggle_metadata = pd.read_csv(f'{kaggle_metadata_csv_file}')
    ratings = pd.read_csv(f'{file_dir}ratings.csv')
    
    # Convert raw to DataFrame
    wiki_movies_df = pd.DataFrame(wiki_movies_raw)
    
    # Clean Wikipedia Movies
    clean_movies = [clean_movie(movie) for movie in wiki_movies]
    wiki_movies_df = pd.DataFrame(clean_movies)
    wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
    print(len(wiki_movies_df))
    wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    print(len(wiki_movies_df))
    wiki_movies_df.head()
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
    wiki_movies_df
    # Make a data series that drops mossing Box office values
    box_office = wiki_movies_df['Box office'].dropna()
    
    
    